In [1]:
#inlezen van datasets met interacties en bijhorende GO en IPR termen
#controleren op entries zonder GO en/of IPR termen
#remappen van deze entries en controle of geremapte ID wel GO en/of IPR termen geeft

In [2]:
import pandas as pd
import numpy as np
import requests
BASE = 'http://www.uniprot.org'
KB_ENDPOINT = '/uniprot/'
TOOL_ENDPOINT = '/uploadlists/'

In [3]:
def map_retrieve(ids2map, source_fmt='ACC+ID',
                 target_fmt='ACC', output_fmt='list'):
    if hasattr(ids2map, 'pop'):
        ids2map = ' '.join(ids2map)
    payload = {'from': source_fmt,
               'to': target_fmt,
               'format': output_fmt,
               'query': ids2map,
               }
    response = requests.get(BASE + TOOL_ENDPOINT, params=payload)
    if response.ok:
        return response.text
    else:
        response.raise_for_status()

In [4]:
species = 'Francisella tularensis'

In [5]:
Int_DB = pd.read_table('/Users/thesis/Desktop/Int_DB_IPR_GO_' + species, usecols=lambda x:x !="Unnamed: 0")

In [6]:
sel1 = Int_DB.IPR_A.isnull()
sel2 = Int_DB.IPR_B.isnull()
sel3 = Int_DB.GO_A.isnull()
sel4 = Int_DB.GO_B.isnull()

In [7]:
listA = Int_DB.loc[sel1|sel3,'Protein_A'].unique()
listB = Int_DB.loc[sel2|sel4,'Protein_B'].unique()
#Int_DB.Protein_A = Int_DB.Protein_A.apply(lambda x: [x])
#Int_DB.Protein_B = Int_DB.Protein_B.apply(lambda x: [x])
Int_DB['A_remapped_from'] =np.nan
Int_DB['B_remapped_from'] = np.nan
#print(listA)
#print(listB)

In [8]:
with open('/Users/thesis/Desktop/multiple_alt_ids_A.txt','w') as w:
    for name in listA:
        sel = Int_DB.Protein_A == name
        Int_DB.loc[sel,'A_remapped_from'] = name
        alt_id = map_retrieve(name)
        Int_DB.loc[sel,'Protein_A'] = map_retrieve(name).strip()
        if len(alt_id.split())>1:
            w.write(name+ '\t' + alt_id + '\n')

In [9]:
with open('/Users/thesis/Desktop/multiple_alt_ids_B.txt','w') as w:
    for name in listB:
        sel = Int_DB.Protein_B == name
        Int_DB.loc[sel,'B_remapped_from'] = name
        alt_id = map_retrieve(name)
        Int_DB.loc[sel,'Protein_B'] = alt_id.strip()
        if len(alt_id.split())>1:
            w.write(name + '\t' + alt_id + '\n')

In [10]:
#inspection of the entries which have been written out to the txt file resulted in the decision to remap:
#P62158 to P0DP25
#O95766 to P86791
#P08107 to P0DMV9

to_remap=['P62158','O95766','P08107']
goal_id=['P0DP25','P86791','P0DMV9']
for i in range (3):
    sel = Int_DB.B_remapped_from == to_remap[i]
    Int_DB.loc[sel,'Protein_B'] = goal_id[i]

In [11]:
Int_DB.to_csv("/Users/thesis/Desktop/Int_DB_IPR_GO_Remap_" + species, sep ="\t")